# Submodule2: Collect and prepare sequence data and analysis:

# Learning Objectives:
In submodule 2 we will leverage the fundamental concepts from [**Submodule 1**](./SubModule1.ipynb) to do the following:
- Demonstrate different ways of extracting dataset
- Preparing those data for phylogenetic analysis
- Discuss how cloud solutions can be helpful on all these things
- Implement Quality Control checks on the dataset

----------------------------------------------------------------------------------------------------------------
# Training Plan 


Submodule #1: Understanding the Basics of Phylogenetic

<font color="green"> **Submodule #2: Collect and Prepare Sequence Data and Analysis** </font>

Submodule #3: Construct Phylogenetic Tree

 
Submodule #4: Analyze Phylogenetic Tree

----------------------------------------------------------------------------------------------------------------

## 2.1 Efficient Methods for Sourcing Pathogen Sequences

Efficient sourcing and preparation of sequence data are critical for accurate phylogenetic analysis.

### Steps to Source Pathogen Sequences
1. **Public Databases:**
   - Access public repositories like **GenBank**, **EMBL**, and **DDBJ** to retrieve high-quality, annotated genetic data.
2. **Automated Sequence Retrieval Tools:**
   - Tools like **Entrez Direct** and **Biopython** simplify sequence retrieval and minimize manual errors.
3. **Preprocessing Tools:**
   - Use tools like **Trimmomatic** or **Cutadapt** to remove low-quality reads and adapters from the sequences.
4. **Sequence Alignment:**
   - Use alignment tools like **MAFFT** or **ClustalW** to align sequences, ensuring proper alignment for downstream analysis.

<div style="padding: 10px; border: 1px solid #b3e5fc; border-radius: 5px; background-color: #e1f5fe;">
    <strong>Tip:</strong>💡 Use advanced search options in public databases to filter results based on organism, region, or gene of interest.
</div>

<div style="padding: 10px; border: 1px solid #b3e5fc; border-radius: 5px; background-color: #e1f5fe;">
    <strong>Note:</strong> 📝 Validate the alignment using metrics like alignment scores to avoid errors during tree construction.
</div>

<div style="padding: 10px; border: 1px solid #ffccbc; border-radius: 5px; background-color: #ffebee;">
    <strong>Alert:</strong>⚠️ Poor preprocessing can lead to errors in alignment and tree construction.
</div>

## 2.2 Importance of Cloud-Based Storage Solutions

Cloud-based storage solutions simplify the management of large-scale sequence data.

### Benefits of Cloud-Based Solutions
1. **Scalability:** Handle large datasets without local storage constraints.
2. **Accessibility:** Enable real-time collaboration by making datasets accessible from any location.
3. **Cost-Effectiveness:** Pay-as-you-go models minimize costs by charging only for used resources.
4. **Data Security and Backup:** Automatic backups and encryption protect datasets from loss or breaches.

<div style="padding: 10px; border: 1px solid #b3e5fc; border-radius: 5px; background-color: #e1f5fe;">
    <strong>Note:</strong> 📝 Consider using established platforms like AWS, Google Cloud, or Azure to streamline data storage and processing.
</div>

<div style="padding: 10px; border: 1px solid #b3e5fc; border-radius: 5px; background-color: #e1f5fe;">
    <strong>Tip:</strong>💡 Leverage genomic analysis services offered by cloud providers for faster processing.
</div>

## 2.3 Enhancing Analysis with Publicly Available Datasets

Publicly available datasets improve the depth and accuracy of phylogenetic analyses.

### Benefits of Public Datasets
1. **Increased Data Volume:** Larger datasets improve statistical power and robustness.
2. **Comparative Analysis:** Analyze trends across species or geographic regions using existing data.
3. **Validation and Reproducibility:** Public datasets enable reproducible research.
4. **Collaborative Research:** Shared datasets foster collaboration and knowledge-sharing among researchers.

<div style="padding: 10px; border: 1px solid #b3e5fc; border-radius: 5px; background-color: #e1f5fe;">
    <strong>Note:</strong> 📝  Verify the format and metadata compatibility of public datasets before integrating them into your analysis pipeline.
</div>

# Create data directory to store data
To manage datasets efficiently, it is essential to organize them into directories. This ensures that all related files are stored systematically and can be accessed without confusion. The following Python code automates this process:

In [ ]:
import os
data_dir = os.path.isdir('./data')
if not data_dir:
    !mkdir data

This code checks if a directory named data exists in the current working directory. If it does not exist, it creates the directory. This is useful for organizing and storing datasets.

## Understanding the SARS-CoV-2 Dataset

The **SARS-CoV-2 dataset** contains genetic sequence data for the virus responsible for the COVID-19 pandemic. This dataset is widely used in phylogenetic studies to track mutations, understand evolutionary relationships, and analyze the spread of the virus globally.

### **Key Components of the SARS-CoV-2 Dataset**
1. **Genetic Sequences:**
   - The dataset typically contains nucleotide or protein sequences for various strains of the virus.
   - These sequences are vital for studying how the virus evolves over time and across different regions.

2. **Metadata:**
   - Associated metadata includes information such as:
     - Geographic location of sample collection.
     - Date of sample collection.
     - Variant classification or lineage (e.g., Delta, Omicron).
   - Metadata enhances analysis by providing context for understanding variations across samples.

3. **Applications:**
   - **Phylogenetic Analysis:** Track how different strains of the virus are related.
   - **Mutation Analysis:** Identify specific mutations and their impacts on viral behavior (e.g., transmissibility or resistance to vaccines).
   - **Epidemiological Studies:** Analyze how the virus spreads in specific populations.

### **Sources of SARS-CoV-2 Data**
1. **Nextstrain Project:**
   - A global initiative providing curated datasets for viral evolution studies.
   - SARS-CoV-2 datasets from Nextstrain are preprocessed for downstream phylogenetic analysis.
   - Visit the [Nextstrain documentation](https://docs.nextstrain.org/projects/nextclade/en/stable/user/datasets.html) for more details.

2. **NCBI Virus Database:**
   - A comprehensive resource for viral sequences and metadata.
   - The NCBI database allows users to search by strain, variant, or geographic location.
   - Access the NCBI SARS-CoV-2 dataset [here](https://www.ncbi.nlm.nih.gov/labs/virus/vssi/#/virus?SeqType_s=Nucleotide&VirusLineage_ss=taxid:2697049&USAState_s=SD).

### **Why Use SARS-CoV-2 Data?**
1. **Scientific Relevance:**
   - SARS-CoV-2 remains a high-priority topic for research due to its global impact.
   - Understanding its genetic evolution helps predict future outbreaks and design effective vaccines.

2. **Real-World Applications:**
   - This dataset is used in vaccine development, drug resistance studies, and public health policies.
   - It is also a benchmark for testing bioinformatics tools in a real-world context.

<div style="padding: 10px; border: 1px solid #b3e5fc; border-radius: 5px; background-color: #e1f5fe;">
    <strong>Tip:</strong>💡 Always cross-check the integrity and annotation quality of sequences before using them in your analysis.
</div>

## Download SARS-Cov-2 Dataset

In [ ]:
import os

# Check if the directory exists
cov_dir = os.path.isdir('./data/cov')

# If the directory does not exist, create it
if not cov_dir:
    try:
        os.makedirs('./data/cov/')
        print("Directory created successfully")
    except Exception as e:
        print(f"An error occurred: {e}")

### NCBI SARS-CoV-2 sequence file.

#### Subset of the data 
The subset of the data use for tutorial (./data/cov/sequence_PQ230960.fasta). The full data require heavy compute resource (./data/cov/sequence.fasta).

### Data Preparation
**Convert the .fasta file to .fastq format using the following Python code:**


In [ ]:
from Bio import SeqIO


def fasta_to_fastq(fasta_file, fastq_file, quality=40):
    with open(fastq_file, "w") as output_handle:
        for record in SeqIO.parse(fasta_file, "fasta"):
            record.letter_annotations["phred_quality"] = [quality] * len(record.seq)
            SeqIO.write(record, output_handle, "fastq")


if __name__ == '__main__':
    fasta_file = "./data/cov/sequences_subset.fasta"
    fastq_file = "./data/cov/sequences_subset_converted.fastq"
    fasta_to_fastq(fasta_file=fasta_file, fastq_file=fastq_file)


This code converts a .fasta file containing SARS-CoV-2 sequence data to .fastq format using Biopython. The fasta_to_fastq function takes a default quality score of 40 for all sequences, which is typical for high-quality sequence data.

## 2.4 Quality Control Checks

Quality control ensures the reliability and accuracy of sequence data.

### Steps for Quality Control
1. **Install and Configure FastQC:**
   - Ensure a **Java Runtime Environment (JRE)** is installed.
   - **Ubuntu Command:** `sudo apt install default-jre`
   - **RedHat Command:** `sudo yum install java-1.8.0-openjdk`
   - Verify installation:
     ```bash
     java --version
     fastqc --version
     ```

2. **Run FastQC:**
   - Analyze the quality of `.fastq` files with the following command:

<div style="padding: 10px; border: 1px solid #ffccbc; border-radius: 5px; background-color: #ffebee;">
    <strong>Alert:</strong>⚠️ Address low-quality sequences and high adapter content before proceeding to sequence alignment.
</div>

### Run FastQC

In [ ]:
!fastqc -t 4 ./data/cov/sequences_subset_converted.fastq

## Interactive Quiz

Test your understanding of sequence data preparation and quality control using this interactive quiz:

In [ ]:
from jupyterquiz import display_quiz
display_quiz('Quiz/QS2.json')

# More Datasets to explore

### KEGG Dataset:
Dataset 1: KEGG for Phylogenetic Tree
Downloading KEGG Dataset
**KEGG (Kyoto Encyclopedia of Genes and Genomes)** provides a wealth of data for understanding high-level functions and utilities of biological systems. To download KEGG data:
1. Access KEGG Dataset: KEGG provides a website for data retrieval.
    - https://www.genome.jp/kegg/seq/
2. Downloaded file from FASTA sequence files section on the website.
    - More resources on KEGG: https://www.genome.jp/kegg/


## Download KEGG dataset from URL

### Create Kegg folder to store data

In [ ]:
import os

# Check if the directory exists
kegg_dir = os.path.isdir('./data/kegg')

# If the directory does not exist, create it
if not kegg_dir:
    try:
        os.makedirs('./data/kegg/')
        print("Directory created successfully")
    except Exception as e:
        print(f"An error occurred: {e}")

## Uniport Dataset
Dataset 2: UniProt for Phylogenetic Tree
Downloading UniProt Dataset
UniProt is a comprehensive resource for protein sequence and functional information. To download UniProt data:
1. Access UniProt Website: Visit the UniProt website and search for the desired protein sequences.
2. Retrieve Data: Downloaded the Isoform sequences fasta file from the website.
    - File name: uniprot_sprot_varsplic.fasta

## Download Uniport Dataset from URL

In [ ]:
import os

# Check if the directory exists
uniport_dir = os.path.isdir('./data/uniport')

# If the directory does not exist, create it
if not uniport_dir:
    try:
        os.makedirs('./data/uniport/')
        print("Directory created successfully")
    except Exception as e:
        print(f"An error occurred: {e}")

# Installations

In [ ]:
!pip install biopython

In [ ]:
!conda install bioconda::fastqc -y